# DataLoader Setup

In [1]:
import os
import torch
import torchvision
from pytorch_lightning import seed_everything
from pl_bolts.datamodules import CIFAR10DataModule, TinyCIFAR10DataModule
from pl_bolts.transforms.dataset_normalizations import cifar10_normalization

PATH_DATASETS = os.environ.get('PATH_DATASETS', '../datasets')
AVAIL_GPUS = min(1, torch.cuda.device_count())
BATCH_SIZE = 512 if AVAIL_GPUS else 64
NUM_WORKERS = int(os.cpu_count()/2)
TEST_ONLY = False

test_transforms = [
        torchvision.transforms.ToTensor(),
        cifar10_normalization(),        
        torchvision.transforms.Lambda(lambda x: x.clamp(min = -1, max = 1)),
        torchvision.transforms.Lambda(lambda x: x * 127),   
        torchvision.transforms.Lambda(lambda x: x.floor())  
]

train_transforms = torchvision.transforms.Compose(
    [
        torchvision.transforms.RandomCrop(32, padding=4),
        torchvision.transforms.RandomHorizontalFlip(),
    ] 
    + test_transforms
)

test_transforms = torchvision.transforms.Compose(test_transforms)

cifar10_dm = CIFAR10DataModule(
    data_dir = PATH_DATASETS,
    batch_size = BATCH_SIZE,
    num_workers = NUM_WORKERS,
    train_transforms = train_transforms,
    test_transforms = test_transforms,
    val_transforms = test_transforms,
)

seed_everything(seed=1234, workers=True)

/home/neungyun/anaconda3/envs/torch/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:73: LightningDeprecationWarning: DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(
/home/neungyun/anaconda3/envs/torch/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:77: LightningDeprecationWarning: DataModule property `val_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(
/home/neungyun/anaconda3/envs/torch/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:81: LightningDeprecationWarning: DataModule property `test_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(
Global seed set to 1234


1234

# model 

In [ ]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
from model import BaselineModel,Base_Normal_Block,Base_Reduction_Block

binary_structure = [{'first'= True, 'in_channels':3, 'block':'reduction', 'conv':'real', 'act_fn':'sign'},
             {'in_channels':128, 'out_channels':128, 'stride':2, 'kernel_size':3, 'padding':1, 'conv':'scaled_sign', 'act_fn':'sign', 'dropout':0},
             {'in_channels':128, 'out_channels':256, 'stride':1, 'kernel_size':3, 'padding':1, 'conv':'scaled_sign', 'act_fn':'sign', 'dropout':0},
             {'in_channels':256, 'out_channels':256, 'stride':2, 'kernel_size':3, 'padding':1, 'conv':'scaled_sign', 'act_fn':'sign', 'dropout':0},
             {'in_channels':256, 'out_channels':512, 'stride':1, 'kernel_size':3, 'padding':1, 'conv':'scaled_sign', 'act_fn':'sign', 'dropout':0},
             {'in_channels':512, 'out_channels':512, 'stride':2, 'kernel_size':3, 'padding':1, 'conv':'scaled_sign', 'act_fn':'sign', 'dropout':0},
             {'in_channels':512, 'out_channels':1024, 'stride':1, 'kernel_size':4, 'padding':0, 'conv':'scaled_sign', 'act_fn':'sign', 'dropout':0.5},
             {'in_channels':1024, 'out_channels':1024, 'stride':1, 'kernel_size':1, 'padding':0, 'conv':'scaled_sign', 'act_fn':'sign', 'dropout':0.5},
             {'in_channels':1024, 'out_channels':10, 'stride':1, 'kernel_size':1, 'padding':0, 'conv':'scaled_sign', 'act_fn':'none', 'dropout':0},]

real_structure = [{'in_channels':3, 'out_channels':128, 'stride':1, 'kernel_size':3, 'padding':1, 'conv':'real', 'act_fn':'relu', 'dropout':0},
             {'in_channels':128, 'out_channels':128, 'stride':2, 'kernel_size':3, 'padding':1, 'conv':'real', 'act_fn':'relu', 'dropout':0},
             {'in_channels':128, 'out_channels':256, 'stride':1, 'kernel_size':3, 'padding':1, 'conv':'real', 'act_fn':'relu', 'dropout':0},
             {'in_channels':256, 'out_channels':256, 'stride':2, 'kernel_size':3, 'padding':1, 'conv':'real', 'act_fn':'relu', 'dropout':0},
             {'in_channels':256, 'out_channels':512, 'stride':1, 'kernel_size':3, 'padding':1, 'conv':'real', 'act_fn':'relu', 'dropout':0},
             {'in_channels':512, 'out_channels':512, 'stride':2, 'kernel_size':3, 'padding':1, 'conv':'real', 'act_fn':'relu', 'dropout':0},
             {'in_channels':512, 'out_channels':1024, 'stride':1, 'kernel_size':4, 'padding':0, 'conv':'real', 'act_fn':'relu', 'dropout':0.5},
             {'in_channels':1024, 'out_channels':1024, 'stride':1, 'kernel_size':1, 'padding':0, 'conv':'real', 'act_fn':'relu', 'dropout':0.5},
             {'in_channels':1024, 'out_channels':10, 'stride':1, 'kernel_size':1, 'padding':0, 'conv':'real', 'act_fn':'none', 'dropout':0},]


In [35]:

channel_array = [3,6,12]
x = torch.randn(100,3,32,32)
layer1 = Base_Normal_Block(3)
layer2 = Base_Reduction_Block(3)
layer3 = Base_Reduction_Block(6)
layer4 = Base_Reduction_Block(12)
layer5 = Base_Reduction_Block(24)
layer6 = Base_Reduction_Block(48)
layer7 = GeneralConv2d




out = layer1(x)
out = layer2(out)
out = layer3(out)
out = layer4(out)
out = layer5(out)


out.shape

torch.Size([100, 48, 2, 2])

In [12]:
base_model(base_model.example_input_array)


NotImplementedError: 

In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

checkpoint_callback = ModelCheckpoint(filename='{epoch}-{val_loss:.4f}-{val_acc:.4f}', monitor='val_acc', mode='max')
trainer = Trainer(
    max_epochs=-1,
    gpus=AVAIL_GPUS,
    logger=TensorBoardLogger('lightning_logs/', name='ReActNet', log_graph=True),
    # logger=TensorBoardLogger('lightning_logs/', name='Real', log_graph=True),
    callbacks=[LearningRateMonitor(logging_interval='step'), 
               EarlyStopping(monitor='val_acc', mode='max', patience=100),
               checkpoint_callback],
    deterministic=True,
#    gradient_clip_val = 0.5
)

trainer.fit(base_model, datamodule=cifar10_dm)